# Prelab: An introduction to the tidyverse (R - II)

In the previous sections, you learned about some functionality of in the 'base' of R. However, there is a much more extensive, useable and useful infrastructure that is available in R that we will use for much of the course (and you should use for your future scientific careers!).

In the following modules, we will be learning the basics of a cluster of R packages collectively known as the ["tidyverse"](https://www.tidyverse.org/). This is a set of tools that makes data manipulation and visualization in R easier and more flexible than in the basic R language. 

This framework encapsulates a number of extremely useful libraries and are coded to allow for their interoperabilty. These packages includ `dplyr()`, `tidyr()`, `readr()`, `tibble()`, and `ggplot2()`, to name a few. The capabilities of these packages are much greater than what we can cover in this module, and there is a list of resources at the end of this prelab to help you continue to learn beyond the basics.

## Loading packages and data

As you will remember from our previous R modules, we need to load the libraries we will be using before we begin any analysis.

In [0]:
library(tidyverse)
options(repr.plot.width=10, repr.plot.height=3) #set size for plots in this notebook

For this prelab, we will be using some cancer incidence statistics from 2014 (obtained from https://www.cdc.gov/cancer/). This dataset contains statistics for a set of seven types of cancer, stratified by year, race, and sex. 

In [0]:
data = read.table("Cancer_Incidence.txt",header=T,sep="\t")
head(data)

## Manipulating data with dplyr

First, we will convert our data from the standard "data frame" format R creates when you use read.table into the "tbl" (table, or sometimes pronounce "tibble") format designed for use in dplyr, using the command `as_tibble()`.

In [0]:
data = as_tibble(data)

A key difference in dplyr from basic R is the "pipe" command (`%>%`). This command let's us string together commands, resulting in clearer and more modular code. Let's combine `%>%` with the command `select()`, which allows use to select a subset of columns from our data frame. Below we take our entire dataset, then select two columns, then pipe that to head to just see the first few rows. Notice how we can string together multiple pipes, with our old friend `head()` at the end to print just the beginning of the tbl:

In [0]:
data %>% select(YEAR,RACE) %>% head()

Note here that we're not storing the result to the existing variable (or to a new one!), but simply doing basic manipulations to the original table. 

**If you want to store the output, use variable assignment! (e.g., `myresult = data %>% ...`)**

We can also use `select()` to remove the SEX and YEAR columns using `-`:

In [0]:
data %>% select(-SEX,-YEAR) %>% head()

Now let's try the `filter()` command, which let's us subset the data by choosing rows which follow the given conditions. Here we subset the data to only look at the female statistics. Note that we use the boolean condition `==` because are evaluating a true/false statement, rather than setting a variable's value.

In [0]:
table(data$SEX) #prints the counts of each unique value in the SEX column
female = data %>% filter(SEX=="Female") #filter to only include rows where the SEX column is "Female"
table(female$SEX) #now we only have females after using filter

The command `arrange()` will sort the tbl by a given column. Let's filter to include only data on females of all races in the year 1999, and look at the most and least common cancers.

In [ ]:
#arrange: for 1999, Females, All Races, sort by rate lowest->highest
data %>% filter(SEX=="Female",YEAR==1999,RACE=="All Races") %>% arrange(AGE_ADJUSTED_RATE) %>% head()
#arrange: for 1999, Females, All Races, sort by rate highest->lowest
data %>% filter(SEX=="Female",YEAR==1999,RACE=="All Races") %>% arrange(desc(AGE_ADJUSTED_RATE)) %>% head()

The `mutate()` command creates a new column by performing a specified calculation on each row. Here we calculate the crude rate for each row by dividing the count by the total population size, and name our new column "NEW_RATE". You can see that the reported CRUDE_RATE is multiplied by 1e5 (so the "RATE" is per 100,000 individuals).

In [0]:
data %>% mutate(NEW_RATE = COUNT/POPULATION) %>% head()

The `group_by()` command allows you subset your data by a variable and calculate any summary statistics (via `summarise()`) for each group. For example, below we remove the "all races" category, then group by sex and calculate the mean of the reported CRUDE_RATE for each sex.

In [ ]:
data %>% filter(RACE=="All Races") %>% group_by(SEX) %>% summarise(meanRate = mean(CRUDE_RATE)) %>% head()

Notice that this string of commands resulted in only 2 columns, because we only grouped by SEX, thus combining all other variables (e.g. SITE, the type of cancer). Let's try grouping by multiple variables. Here we don't use `head()`, and will get the whole table as output:

In [0]:
#group by sex and site
data %>% filter(RACE=="All Races") %>% group_by(SEX,SITE) %>% summarise(meanRate = mean(CRUDE_RATE))

## A couple of other tidyverse functions that are helpful

There are a couple other functions that we'll use in the course that we want to introduce to you here that are helpful.

**Note: As above, we're not storing the result to the existing variable (or to a new one!) - just doing basic manipulations to the original table. Use variable assignment if you want to save the output! (e.g., `myresult = data %>% ...`)**

The first is `rename()`: this function allows you to rename one of your headers.

In [ ]:
data %>% rename(AGE_ADJ_CI_LOW = AGE_ADJUSTED_CI_LOWER)

Second, is `relocate()`: this function allows you to change the order of columns in your table. The input is the list of column names you want to shift (even as a block or individually), then use either `.before=` or `.after=` to define the column that the block will be placed before or after, respectively:

In [ ]:
data %>% head()
data %>% relocate(AGE_ADJUSTED_RATE, .before=AGE_ADJUSTED_CI_LOWER) #shift the rate to list in front of the confidence interval
data %>% relocate(YEAR, RACE, SEX, .after=SITE) #shift the block to list after site

There are a number of table merging operations that are handy. These include `left_join()`, `right_join()`, `inner_join()`, and `full_join()`.

These merge a "left" table (listed on the left, called 'x') to the table on the "right" (provided on the right, called 'y') in various ways and rules, using `.by` as the key. Briefly:

`left_join()`: includes all rows in x.

`right_join()`: includes all rows in y.

`inner_join()`: includes all rows in x **and** y.

`full_join()`: includes all rows in x **or** y.

Let's make a table or two to demonstrate.

In [ ]:
TISSUE_GCK <- c("Liver", "Bladder", "Pancreas")
GEXP_GCK <- c(5000,5,10)

TISSUE_HNF4A <- c("Liver", "Bladder", "Toenail")
GEXP_HNF4A <- c(1000,10,0)

left_table <- tibble(TISSUE_GCK, GEXP_GCK) %>% rename(TISSUE = TISSUE_GCK)
right_table <- tibble(TISSUE_HNF4A, GEXP_HNF4A) %>% rename(TISSUE = TISSUE_HNF4A)

In [ ]:
#matches right contents with left by the key TISSUE
#thus, Toenail is dropped and NA for GEXP_HNF4A in Pancreas since not present
left_join(left_table, right_table, by="TISSUE") 

In [ ]:
#you can also use pipes for these too! what is piped into the function is "left" by default:
#thus, same as the above
left_table %>% left_join(right_table, by="TISSUE")

In [ ]:
#matches left contents with right by the key TISSUE
#thus, pancreas is dropped and NA for GEXP_GCK in Toenail since not present
right_join(left_table, right_table, by="TISSUE") 

In [ ]:
#matches the intersection of left and right tables by the key TISSUE
#thus, save the Liver and Bladder content since they are the only ones that are common in both tables
#note that you'd get the same result if you swapped the order of tables provided as arguemnts
inner_join(left_table, right_table, by="TISSUE")

In [ ]:
#includes all contents and merge by the key TISSUE
#thus, Liver, Bladder, Pancreas, and Toenail are represented in a merged table with NA as missing accordingly
#note that you'd get the same result if you swapped the order of tables provided as arguemnts
full_join(left_table, right_table, by="TISSUE")

Sometimes, you will want to execute function for each row, rather than on columns. For that, the ecclectic `rowwise()` grouping function can be used where each group consists of a single row.

In [ ]:
df <- tibble(x = 1:2, y = 3:4, z = 5:6)
df %>% rowwise() # note the grouping specified here

# Note the output of m. This is the mean of of x, y, and z tabulated across all rows! 
# Perhaps not what you want
df %>% mutate(m = mean(c(x, y, z)))

# If you want to calculate mean by "grouping" on the row, use rowwise()
# ah, the result is what I was looking for
df %>% rowwise() %>% mutate(m = mean(c(x, y, z)))

There are some handy helper functions that you can read about with this function (e.g., use of `c_across()` for example): 

https://dplyr.tidyverse.org/articles/rowwise.html